## Import Dependencies

In [1]:
# Environment at time of execution
%load_ext watermark
%pylab inline
%watermark -a "Anthony Abercrombie" -d -t -v -p numpy,pandas,matplotlib -g

Populating the interactive namespace from numpy and matplotlib
Anthony Abercrombie 2017-01-28 22:28:47 

CPython 3.5.2
IPython 5.1.0

numpy 1.11.2
pandas 0.19.2+0.g825876c.dirty
matplotlib 1.5.1
Git hash: 53bab6447d281b5d6078325090bb0df93da9dddd


In [2]:
from __future__ import print_function

import os

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import dotenv

In [6]:
import os
import sys
import dotenv
import subprocess
import glob
from tqdm import tqdm

#File path to get to the project root
PROJ_ROOT = os.path.join(os.path.pardir, os.pardir)
# add local python functions
sys.path.append(os.path.join(PROJ_ROOT, "src"))

#Load AWS keys as environment variables
dotenv_path = os.path.join(PROJ_ROOT, '.env')
dotenv.load_dotenv(dotenv_path)

#Get environment variables
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
ec2_keypair = os.environ.get("ec2_keypair")
ec2_keypair_pem = os.environ.get("ec2_keypair_pem")
ec2_ssh = os.environ.get("ec2_ssh")

from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Spin up an EC2 Instance that will process the data

In [10]:
def spinup_ec2(ec2_keypair_pem, ec2_ssh):
    bash_cmd = 'ssh -i {} {}'.format(ec2_keypair_pem, ec2_ssh)
    return bash_cmd
ec2_ssh_cmd = spinup_ec2(ec2_keypair_pem, ec2_ssh)


### SCP requirements.txt file into EC2 instance

In [17]:
def scp_file(ec2_keypair_pem, filepath, ec2_ssh):
    scp_cmd = 'scp -i {} {} {}:'.format(ec2_keypair_pem, filepath, ec2_ssh)
    return scp_cmd
requirements_filepath = PROJ_ROOT+'/requirements.txt'
scp_reqs_cmd = scp_file(ec2_keypair_pem, requirements_filepath, ec2_ssh)

In [18]:
scp_reqs_cmd

'scp -i ~/.ssh/dstl_satellite_imagery.pem ../../requirements.txt ubuntu@ec2-35-162-133-16.us-west-2.compute.amazonaws.com:'

In [ ]:
#basic updates
update1 = 'sudo apt-get update'
install_python = 'sudo apt-get install python-dev python-pip'
create_data_folder = 'mkdir data'
create_data_raw_folder = 'mkdir raw'

In [19]:
#Establish S3 Connection

In [20]:
import boto
from boto.s3.key import Key
from boto.s3.connection import S3Connection

In [21]:
s3 = S3Connection()
dscl_satellite_imagery = s3.get_bucket('dstl-satellite-imagery')

In [22]:
data_bucket = dscl_satellite_imagery.lookup?

In [23]:
data_bucket = dscl_satellite_imagery.get_key?

In [ ]:
data_bucket = dscl_satellite_imagery.get_key

In [24]:
file_list = ['grid_sizes.csv.zip', 'sample_submission.csv.zip', 'sixteen_band.zip', 'three_band.zip', 'train_geojson_v3.zip', 'train_wkt_v4.csv.zip']

In [29]:
files = dscl_satellite_imagery.get_all_keys()


In [30]:
files

[<Key: dstl-satellite-imagery,Users/AnthonyAbercrombie/projects/satellite_imagery_feature_detection/data/raw/grid_sizes.csv.zip>,
 <Key: dstl-satellite-imagery,Users/AnthonyAbercrombie/projects/satellite_imagery_feature_detection/data/raw/sample_submission.csv.zip>,
 <Key: dstl-satellite-imagery,Users/AnthonyAbercrombie/projects/satellite_imagery_feature_detection/data/raw/sixteen_band.zip>,
 <Key: dstl-satellite-imagery,Users/AnthonyAbercrombie/projects/satellite_imagery_feature_detection/data/raw/three_band.zip>,
 <Key: dstl-satellite-imagery,Users/AnthonyAbercrombie/projects/satellite_imagery_feature_detection/data/raw/train_geojson_v3.zip>,
 <Key: dstl-satellite-imagery,Users/AnthonyAbercrombie/projects/satellite_imagery_feature_detection/data/raw/train_wkt_v4.csv.zip>]

In [33]:
grid_sizes = files[0]

In [46]:
import gzip

In [54]:
type(grid_sizes.get_contents_as_string())

bytes

In [56]:
grid_sizes_bytes = grid_sizes.read()
grid_sizes_decoded = grid_sizes_bytes.decode('utf-8')

In [62]:
grid_sizes.key

'Users/AnthonyAbercrombie/projects/satellite_imagery_feature_detection/data/raw/grid_sizes.csv.zip'

In [59]:
with gzip.GzipFile(fileobj=grid_sizes_decoded, mode='rb') as f:
    df = pd.read_csv(f, sep='\t')

AttributeError: 'str' object has no attribute 'read'